# Photo Collecting
open camera

In [1]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224)
camera.running = True

# Timing Collecting

In [2]:
import torchvision.transforms as transforms


TakingPhotoTransforms = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [3]:
import time
import cv2
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import os
import subprocess
import uuid
from jupyter_clickable_image_widget import ClickableImageWidget
import glob
import torch.nn.functional as F
import torchvision
import torch
import torch.optim as optim

camera.unobserve_all()

SamplingNumber = 500#采样数量
SamplingInterval = 0.1#采样间隔时间


StartCollecting_widget = ipywidgets.ToggleButtons(options=['continue', 'stop'], description='state', value='stop')#可视化处理，点击continue开始采样，stop状态下结束采样


category_dir = os.path.join('collect_photo_A','v1')#图片保存路径
def start_takingphoto(change):#上述可视化操作绑定事件
    if change['new'] == 'continue':
        if not os.path.exists(category_dir):
            subprocess.call(['mkdir', '-p', category_dir])#没有路径时创建路径
        for i in range(SamplingNumber):
            filename = '%d_%s.jpg' % (i, str(uuid.uuid1()))#先不添加标签，uuid()为防止重名用来生成随机字符串的python内置函数
            image_path = os.path.join(category_dir, filename)
            cv2.imwrite(image_path, camera.value)
            time.sleep(SamplingInterval)#采样间隔
        StartCollecting_widget.value = 'stop'#采样达到采样数量结束采样

StartCollecting_widget.observe(start_takingphoto,names = 'value')#事件绑定

display(StartCollecting_widget)

ToggleButtons(description='state', index=1, options=('continue', 'stop'), value='stop')

# close camera

In [6]:
camera.running = False
camera.cap.release()

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/jetcam-0.0.0-py3.6.egg/jetcam/camera.py", line 34, in _capture_frames
    self.value = self._read()
  File "/usr/local/lib/python3.6/dist-packages/jetcam-0.0.0-py3.6.egg/jetcam/csi_camera.py", line 40, in _read
    raise RuntimeError('Could not read image from camera')
RuntimeError: Could not read image from camera



# add label

In [8]:
import shutil
import time
import cv2
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import os
import subprocess
import uuid
from jupyter_clickable_image_widget import ClickableImageWidget
import glob

category_dir = os.path.join('collect_photo_A','v1')

image_widget = ClickableImageWidget(width=camera.width, height=camera.height)
marked_widget = ipywidgets.Image(format='jpeg', width=224, height=224)

SwitchOrDelete_widget = ipywidgets.ToggleButtons(options=['next', 'delete','save'], description='state', value=None)#当前状态有下一张图片，删除图片，保存图片

speedUp_sign = ipywidgets.IntSlider(min=-100, max=100, step=1, description='speed_sign')#速度标签
over_sign = ipywidgets.IntSlider(min=0, max=3, step=1, description='over_sign')#超车标签
stop_sign = ipywidgets.IntSlider(min=0, max=10, step=1, description='stop_sign')#停车标签
direction_sign = ipywidgets.IntSlider(min=-100, max=100, step=1, description='direction_sign')#直行还是转向标签


MarkedSpeedUp_sign = ipywidgets.Text(value=str(0),placeholder='Type something',description='label:Speed')#已标速度标签
Markedover_sign = ipywidgets.Text(value=str(0),placeholder='Type something',description='label:over')#已标超车标签
MarkedStop_sign = ipywidgets.Text(value=str(0),placeholder='Type something',description='label:Stop')#已标停车标签
MarkedDirection_sign = ipywidgets.Text(value=str(0),placeholder='Type something',description='label:Direction')#已标直行还是转向标签

collected_image = glob.glob(os.path.join(category_dir,'*.jpg'))#获取前面自动采集图片

point_x = 0
point_y = 0

def marked_snapshot(_, content, msg):#标记图像中x,y坐标函数
    global point_x
    global point_y
    if content['event'] == 'click':
        data = content['eventData']
        point_x = data['offsetX']
        point_y = data['offsetY']
        
        # display saved snapshot
        snapshot = cv2.imread(collected_image[i_picture])
        snapshot = cv2.circle(snapshot, (point_x, point_y), 8, (0, 255, 0), 3)
        marked_widget.value = bgr8_to_jpeg(snapshot)
        
image_widget.on_msg(marked_snapshot)

PresentImage = open(collected_image[0],"rb").read()
image_widget.value = PresentImage



i_picture = 0

#添加标签后的path
category_path= os.path.join('collect_photo_B','v1')

def nextimage(change):#要绑定的图片保存删除下一张等操作
    global i_picture

    basename = os.path.basename(collected_image[i_picture])
    items = basename.split('_')
    x = items[0]
    y = items[1]
    if change['new']=='next':
        i_picture += 1
        PresentImage = open(collected_image[i_picture],"rb").read()
        image_widget.value = PresentImage
    elif change['new']=='delete':
        os.remove(collected_image[i_picture])
    elif change['new']=='save':
        fileName = '%d_%d_%d_%d_%d_%d_%s.jpg' % (int(x), int(y), int(MarkedSpeedUp_sign.value),int(Markedover_sign.value),int(MarkedStop_sign.value),int(MarkedDirection_sign.value),str(uuid.uuid1()))
        RenewName = os.path.join(category_path,fileName)
        os.rename(collected_image[i_picture],RenewName)
        #shutil.copyfile(collected_image[i_picture],RenewName)
        
    time.sleep(1)
    SwitchOrDelete_widget.value = None

SwitchOrDelete_widget.observe(nextimage,names='value')

def updata_MarkedSpeedUp(b):#更新已标速度标签值
    MarkedSpeedUp_sign.value = str(speedUp_sign.value)

def updata_Markedover(b):#更新已标超车标签值
    Markedover_sign.value = str(over_sign.value)

def updata_MarkedStop(b):#更新已标停车标签值
    MarkedStop_sign.value = str(stop_sign.value)

def updata_MarkedDirection(b):#更新已标直行还是转向标签值
    MarkedDirection_sign.value = str(direction_sign.value)

speedUp_sign.observe(updata_MarkedSpeedUp)
over_sign.observe(updata_Markedover)
stop_sign.observe(updata_MarkedStop)
direction_sign.observe(updata_MarkedDirection)


handle_widget = ipywidgets.VBox([
    ipywidgets.HBox([image_widget]),
   ipywidgets.HBox([speedUp_sign,MarkedSpeedUp_sign]),
    ipywidgets.HBox([over_sign,Markedover_sign]),
    ipywidgets.HBox([stop_sign,MarkedStop_sign]),
    ipywidgets.HBox([direction_sign,MarkedDirection_sign]),
    SwitchOrDelete_widget
])

display(handle_widget)

In [5]:
from my_dataset import MY_Dataset
racer_datasets = MY_Dataset(category_path,TakingPhotoTransforms,random_hflip=True)



# Model

### Create data loaders to load data in batches

In [6]:


train_loader = torch.utils.data.DataLoader(
    racer_datasets,
    batch_size = 16,
    shuffle = True,
    num_workers = 4
)

### Define Neural Network Model 

In [7]:
model = torchvision.models.resnet18(pretrained=True)


# 第一个参数是全连接层的数量，第二个参数是输出层的个数
model.fc = torch.nn.Linear(512, 6)

device = torch.device('cuda')
model = model.to(device)

racer_model_path = 'road_following_model.pth'
model_save_button = ipywidgets.Button(description='save model')
model_load_button = ipywidgets.Button(description='load model')
model_path_widget = ipywidgets.Text(description='model path', value=racer_model_path)

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
model_load_button.on_click(load_model)
    
def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
model_save_button.on_click(save_model)

model_widget = ipywidgets.VBox([
    model_path_widget,
    ipywidgets.HBox([model_load_button, model_save_button])
])



In [8]:
NUM_EPOCHS = 20
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

epochs_widget = ipywidgets.IntText(description='epochs', value=1)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')


def train_eval(is_training):
    global model
    
    train_button.disabled = True
    eval_button.disabled = True
    time.sleep(1)
        
    if is_training:
        model = model.train()
    else:
        model = model.eval()
        
    while epochs_widget.value > 0:
        i = 0
        sum_loss = 0.0
        for images, labels in iter(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            if is_training:
                # 参数的零梯度
                optimizer.zero_grad()

            # 执行模型以获得输出
            outputs = model(images)
            loss = F.mse_loss(outputs, labels)
            if is_training:
                # 运行反向传播以累积梯度
                loss.backward()

                # 调整参数的步进优化器
                optimizer.step()
                
            i += 1
            sum_loss += float(loss)
            progress_widget.value = i / len(racer_datasets)
            loss_widget.value = sum_loss / i
                
        if is_training:
            epochs_widget.value = epochs_widget.value - 1
        else:
            break

    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False


train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))

train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    ipywidgets.HBox([train_button, eval_button])
])

display(train_eval_widget,model_widget)
